In [5]:
import pandas as pd
from datetime import datetime
from collections import defaultdict, deque

# Charger le fichier
fichier = "Examen Faculté.xlsx"  # Assurez-vous que ce fichier est dans le même dossier
df_examens = pd.read_excel(fichier, sheet_name="programmation_examens")
df_surveillants = pd.read_excel(fichier, sheet_name="liste_surveillant")

# Préparation des colonnes
df_examens['DATE'] = pd.to_datetime(df_examens['DATE'])
df_examens['HEURE'] = pd.to_datetime(df_examens['HEURE'], format='%H:%M:%S').dt.time
df_examens['DATE_ET_HEURE'] = pd.to_datetime(df_examens['DATE'].astype(str) + ' ' + df_examens['HEURE'].astype(str))

# Détection des créneaux
exam_slots = df_examens[['DATE_ET_HEURE', 'LOCAL']].drop_duplicates().reset_index(drop=True)

def get_required_surveillants(local):
    return 3 if 'AMPHI' in local.upper() else 2

exam_slots['REQUIRED'] = exam_slots['LOCAL'].apply(get_required_surveillants)

# Liste des surveillants
surveillants = df_surveillants['NOM ET PRENOM'].dropna().tolist()
surveillant_queue = deque(surveillants)
surveillance_plan = defaultdict(list)

# Identifier les responsables
responsables = df_examens[df_examens['QUALITE'].str.contains('RESPONSABLE', na=False)][['NOM ET PRENOM', 'DATE']]
responsables_by_day = responsables.groupby('NOM ET PRENOM')['DATE'].apply(set).to_dict()

# Distribution
affectations = []
for _, row in exam_slots.iterrows():
    date_heure = row['DATE_ET_HEURE']
    local = row['LOCAL']
    n_required = row['REQUIRED']
    
    assigned = []
    tries = 0
    while len(assigned) < n_required and tries < len(surveillant_queue) * 2:
        surveillant = surveillant_queue.popleft()
        tries += 1

        if surveillant in responsables_by_day and date_heure.date() in responsables_by_day[surveillant]:
            surveillant_queue.append(surveillant)
            continue
        
        assigned.append(surveillant)
        surveillance_plan[surveillant].append(date_heure)
        surveillant_queue.append(surveillant)
    
    affectations.append({
        'DATE_ET_HEURE': date_heure,
        'LOCAL': local,
        'SURVEILLANTS': assigned
    })

# Intégration dans le tableau original
surveillance_dict = defaultdict(list)
for item in affectations:
    key = (item['DATE_ET_HEURE'], item['LOCAL'])
    surveillance_dict[key] = item['SURVEILLANTS']

def get_surveillants(row):
    key = (pd.to_datetime(row['DATE_ET_HEURE']), row['LOCAL'])
    return ', '.join(surveillance_dict.get(key, []))

df_examens['SURVEILLANTS'] = df_examens.apply(get_surveillants, axis=1)

# Résumé des heures
df_summary = pd.DataFrame([
    {"NOM ET PRENOM": s, "NOMBRE_HEURES": len(h)} for s, h in surveillance_plan.items()
]).sort_values(by="NOMBRE_HEURES", ascending=False)

# Exporter
output = "Répartition_Surveillants_Examen.xlsx"
with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
    df_examens.to_excel(writer, sheet_name="programmation_examens", index=False)
    df_surveillants.to_excel(writer, sheet_name="liste_surveillant", index=False)
    df_summary.to_excel(writer, sheet_name="Résumé_Surveillants", index=False)

print(f"✅ Fichier généré : {output}")
print("\n📋 Répartition complète des surveillants et responsables :\n")

# Regrouper les responsables par (DATE, HEURE, LOCAL)
responsable_map = df_examens[df_examens['QUALITE'].str.contains("RESPONSABLE", na=False)]
responsable_map = responsable_map.groupby(['DATE_ET_HEURE', 'LOCAL'])['NOM ET PRENOM'].first().to_dict()

for a in affectations:
    date_str = a['DATE_ET_HEURE'].strftime("%Y-%m-%d %H:%M")
    local = a['LOCAL']
    surveillants = ', '.join(a['SURVEILLANTS'])
    responsable = responsable_map.get((a['DATE_ET_HEURE'], local), "Aucun")
    
    print(f"{date_str} | {local:<10} → 🧍 Responsable : {responsable} | 👥 Surveillants : {surveillants}")




✅ Fichier généré : Répartition_Surveillants_Examen.xlsx

📋 Répartition complète des surveillants et responsables :

2025-05-14 09:00 | AMPHI A    → 🧍 Responsable : ABIR Abouissaba | 👥 Surveillants : JAZOULI ABDERRAHMANE, AGHBALOU HAMZA, OUAZIK BRAHIM
2025-05-14 09:00 | AMPHI B    → 🧍 Responsable : ABIR Abouissaba | 👥 Surveillants : ZAKARIA ELKHATABI, ELIMANI ABDELHAY, Rafik Abdellah
2025-05-14 09:00 | AMPHI C    → 🧍 Responsable : OU-YASSINE ALI | 👥 Surveillants : ABDELHAFID FROUDIS, OUBOUDIB YOUB, ZENNOUHI MOHAMED
2025-05-14 09:00 | AMPHI S    → 🧍 Responsable : AKHLIL KHALID | 👥 Surveillants : OUZAHIM FATIMA, EL FARKANE LHOUSSAINE, ZIDANE MOHAMMED
2025-05-14 09:00 | SALLE S01  → 🧍 Responsable : AIT OUARASSE OTMANE | 👥 Surveillants : FADILI AHMED, GUILMI REDOUANE
2025-05-14 09:00 | SALLE S02  → 🧍 Responsable : DARMANE YASSINE | 👥 Surveillants : BOUKDIR MOHAMED, ALAOUI MAMOUNI SMAIL
2025-05-14 11:00 | AMPHI A    → 🧍 Responsable : YOUNESS Chawki | 👥 Surveillants : OUDAANI JAOUAD, HADRI AI